<a href="https://colab.research.google.com/github/Min0627/Data-Science/blob/main/P158145__Week10_DataAggregation_20250621.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 10: Data Aggregation and Group Operation**
Please refer to the textbook: **"Python for Data Analysis"** by Wes McKinney for details of this topic on **Chapter 10**

# **Start by importing NumPy and pandas**

In [4]:
import numpy as np
import pandas as pd

# **11.1 Group Operations**
## The popular ***split-apply-combine***

In [6]:
# Example DataFrame
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : rng.standard_normal(7),
                   "data2" : rng.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [9]:
# Access data1 and call groupby with the key1 column
# Create an intermediate file
grouped = df["data1"].groupby(df["key1"])

In [10]:
# Compute median
grouped.max()

,data1
key1,
a,1.263728
b,-0.075343


### **Without intermediate grouped object**

In [11]:
# Performs the same operations but in a single step
df["data1"].groupby(df["key1"]).max()

,data1
key1,
a,1.263728
b,-0.075343


In [13]:
# Passing multiple groupby arrays of keys to find mean
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -1.423825
      2       1.263728
b     1      -0.075343
      2      -0.259173
Name: data1, dtype: float64

In [14]:
# Unstack: from vertical to wide table
means.unstack()

key2,1,2
key1,,
a,-1.423825,1.263728
b,-0.075343,-0.259173


In [15]:
# Pass column names as the group keys
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.300327,0.083521
b,1.5,-0.167258,1.657953


In [16]:
# Groupby using multiple keys
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -1.423825  0.648893
     2     1.263728  0.361058
b    1    -0.075343  0.968497
     2    -0.259173  2.347410

In [17]:
# Returns a Series containing group sizes
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [18]:
# Missing values in a group key are excluded from the result by default
# Can be disabled by passing dropna=False to groupby
# dropna=True is by default
df.groupby("key1").size()

,0
key1,
a,3
b,2


In [32]:
# Using dropna=False
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [20]:
# Can use count as well
df.groupby(["key1", "key2"])["data1"].count()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
Name: data1, dtype: int64

In [21]:
# Returns a Series containing group sizes
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

### **`count()`**: Returns the number of non-null values in each group for each column.

In [22]:
# Difference between size() and count()
import pandas as pd
import numpy as np

df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})

print("DataFrame:")
print(df)
print("\nsize():")
print(df.groupby("key1").size())
print("\ncount():")
print(df.groupby("key1").count())

DataFrame:
   key1  key2     data1     data2
0     a     1  1.533211  1.249192
1     a     2  0.060067  0.085010
2  None     1 -0.771839  1.146969
3     b     2 -0.478917  1.682176
4     b     1 -0.388336  0.683792
5     a  <NA>  0.956856  2.019301
6  None     1 -0.454332  0.902804

size():
key1
a    3
b    2
dtype: int64

count():
      key2  data1  data2
key1                    
a        2      3      3
b        2      2      2


### **Iterating over Groups**
***groupby*** supports ***iteration***, generating ***a sequence of 2-tuples***

In [23]:
# Display df content
df

,key1,key2,data1,data2
0,a,1,1.533211,1.249192
1,a,2,0.060067,0.085010
2,None,1,-0.771839,1.146969
3,b,2,-0.478917,1.682176
4,b,1,-0.388336,0.683792
5,a,<NA>,0.956856,2.019301
6,None,1,-0.454332,0.902804


# **`naMa` and `Grp` is a placeholder**
- holding some information

In [70]:
# Iteration
for naMa, Grp in df.groupby("key1"):
    print(naMa)
    print(Grp)

a
  key1  key2     data1     data2
0    a     1  1.533211  1.249192
1    a     2  0.060067  0.085010
5    a  <NA>  0.956856  2.019301
b
  key1  key2     data1     data2
3    b     2 -0.478917  1.682176
4    b     1 -0.388336  0.683792


In [73]:
# Computing a dictionary using one-liner
pieces = {name: group for name, group in df.groupby("key1")}

In [74]:
# Accessing item in dictionary
pieces["a"]

,key1,key2,data1,data2
0,a,1,1.533211,1.249192
1,a,2,0.060067,0.085010
5,a,<NA>,0.956856,2.019301


In [69]:
# Accessing item in dictionary
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.478917,1.682176
4,b,1,-0.388336,0.683792


### **Selecting a Column or Subset of Columns**

In [78]:
# Returned a DataFrame if a list or array is passed
# If use double square bracket, the output would be a dataframe
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     1.249192
     2     0.085010
b    1     0.683792
     2     1.682176

In [79]:
# Returned a grouped Series if only column name is passed as a scalar
# scalar -> a single column name string
df.groupby(["key1", "key2"])["data2"].mean()

key1  key2
a     1       1.249192
      2       0.085010
b     1       0.683792
      2       1.682176
Name: data2, dtype: float64

### **Grouping with Dictionaries and Series**

In [80]:
# Example DataFrame
# Setting the seed number to ensure reproducibility
rng = np.random.default_rng(seed=12345)
people = pd.DataFrame(rng.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [ ]:
# Add a few NA values using loc function
people.loc["Wanda", ["b", "c"]] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,NaN,NaN,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [81]:
# Add a few NA values using iloc function
people.iloc[(4, 1), (0, 3)] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189


In [87]:
# Group correspondence for the columns
# Act as a dictionary
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange", "g" : "violet"}

In [88]:
# Using the mapping function
people.T.groupby(mapping).sum().T

,blue,orange,red,violet
Joe,-1.129835,-1.423825,-0.235440,-0.740885
Steve,0.648893,1.263728,-4.061540,-1.367793
Wanda,0.142811,-0.870662,2.848953,0.648893
Jill,-0.680811,-0.259173,2.127134,0.361058
Trey,-0.718663,-0.075343,-0.457888,-1.952863


In [89]:
# Add the column f and g
rng = np.random.default_rng(seed=12345)
people = people.assign(f = rng.standard_normal((5, 1)),
                       g = rng.standard_normal((5, 1)))
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863,1.263728,-1.367793
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [90]:
# Run sum function again
people.T.groupby(mapping).sum().T

,blue,orange,red,violet
Joe,-1.129835,-1.423825,-0.235440,-0.740885
Steve,0.648893,1.263728,-4.061540,-1.367793
Wanda,0.142811,-0.870662,2.848953,0.648893
Jill,-0.680811,-0.259173,2.127134,0.361058
Trey,-0.718663,-0.075343,-0.457888,-1.952863


### **Grouping with Functions**

In [91]:
# Display people content
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863,1.263728,-1.367793
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [92]:
# Groupby length of the index letter
people.groupby(len).sum()

,a,b,c,d,e,f,g
3,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
4,-0.060690,0.489146,-0.353749,-1.045725,1.240790,-0.334517,-1.591805
5,1.606525,-0.399296,-0.110494,0.902198,-2.419816,0.393067,-0.718900


# **11.2 Data Aggregation**

In [93]:
# From previous example
df

,key1,key2,data1,data2
0,a,1,1.533211,1.249192
1,a,2,0.060067,0.085010
2,None,1,-0.771839,1.146969
3,b,2,-0.478917,1.682176
4,b,1,-0.388336,0.683792
5,a,<NA>,0.956856,2.019301
6,None,1,-0.454332,0.902804


In [99]:
# Selects the smallest requested number
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     1    0.060067
      5    0.956856
b     3   -0.478917
      4   -0.388336
Name: data1, dtype: float64

In [100]:
# Define own aggregation functions
def peak_2_peak(arr):
    return arr.max() - arr.min()

In [103]:
# agg: aggregate
# apply the function to the dataframe
grouped.agg(peak_2_peak)

,key2,data1,data2
key1,,,
a,1,1.473144,1.934291
b,1,0.090581,0.998384


### **`describe()`**: generates **descriptive statistics** for each group in the grouped object.
- provides **a summary of the data**

In [104]:
# Another way
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.850045  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.433627  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     1.245033  1.533211   3.0  1.117835  0.973813  0.085010  0.667101   
b    -0.410981 -0.388336   2.0  1.182984  0.705964  0.683792  0.933388   

                                    
           50%       75%       max  
key1                                
a     1.249192  1.634247  2.019301  
b     1.182984  1.432580  1.682176  

[2 rows x 24 columns]

## **Column-Wise and Multiple Function Application**

In [105]:
# Get the tipping dataset from github
# Download here: https://bit.ly/3VyE0vP
tips = pd.read_csv("https://bit.ly/3VyE0vP")
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [106]:
# Add additional column
tips["tip_pct"] = tips["tip"] / (tips['total_bill'] - tips['tip'])
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886
3,23.68,3.31,No,Sun,Dinner,2,0.162494
4,24.59,3.61,No,Sun,Dinner,4,0.172069
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.256166
240,27.18,2.00,Yes,Sat,Dinner,2,0.079428
241,22.67,2.00,Yes,Sat,Dinner,2,0.096759
242,17.82,1.75,No,Sat,Dinner,2,0.108899


In [107]:
# Group the tips by day and smoker
grouped = tips.groupby(["day", "smoker"])

In [108]:
# Get descriptive statistics
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.179740
      Yes       0.216293
Sat   No        0.190412
      Yes       0.179833
Sun   No        0.193617
      Yes       0.322021
Thur  No        0.193424
      Yes       0.198508
Name: tip_pct, dtype: float64

In [109]:
# Getting back a DataFrame
grouped_pct.agg(["mean", "std", peak_2_peak])

mean       std  peak_2_peak
day  smoker                                 
Fri  No      0.179740  0.039458     0.094263
     Yes     0.216293  0.077530     0.242219
Sat  No      0.190412  0.058626     0.352192
     Yes     0.179833  0.089496     0.446137
Sun  No      0.193617  0.060302     0.274897
     Yes     0.322021  0.538061     2.382107
Thur No      0.193424  0.056065     0.284273
     Yes     0.198508  0.057170     0.219047

In [110]:
# Another example
grouped_pct.agg([("purata", "mean"), ("std.dev", "std")])

purata   std.dev
day  smoker                    
Fri  No      0.179740  0.039458
     Yes     0.216293  0.077530
Sat  No      0.190412  0.058626
     Yes     0.179833  0.089496
Sun  No      0.193617  0.060302
     Yes     0.322021  0.538061
Thur No      0.193424  0.056065
     Yes     0.198508  0.057170

In [112]:
# Specify a list of functions to apply
functions = ["count", "mean", "max", "min", "median"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                                         total_bill  \
              count      mean       max       min    median      count   
day  smoker                                                              
Fri  No           4  0.179740  0.231125  0.136861  0.175487          4   
     Yes         15  0.216293  0.357737  0.115518  0.210526         15   
Sat  No          45  0.190412  0.412409  0.060217  0.176681         45   
     Yes         42  0.179833  0.483092  0.036955  0.181513         42   
Sun  No          57  0.193617  0.338101  0.063204  0.192841         57   
     Yes         19  0.322021  2.452381  0.070274  0.160256         19   
Thur No          45  0.193424  0.362976  0.078704  0.181324         45   
     Yes         17  0.198508  0.317965  0.098918  0.181818         17   

                                              
                  mean    max    min  median  
day  smoker                                   
Fri  No      18.420000  22.75  12.46  19.235  
     Yes     16.813333  40.17   5.75  13.420  
Sat  No      19.661778  48.33   7.25  17.820  
     Yes     21.276667  50.81   3.07  20.390  
Sun  No      20.506667  48.17   8.77  18.430  
     Yes     24.120000  45.35   7.25  23.100  
Thur No      17.113111  41.19   7.51  15.950  
     Yes     19.190588  43.11  10.34  16.470

In [113]:
# Extract certain column
result["total_bill"]

count       mean    max    min  median
day  smoker                                        
Fri  No          4  18.420000  22.75  12.46  19.235
     Yes        15  16.813333  40.17   5.75  13.420
Sat  No         45  19.661778  48.33   7.25  17.820
     Yes        42  21.276667  50.81   3.07  20.390
Sun  No         57  20.506667  48.17   8.77  18.430
     Yes        19  24.120000  45.35   7.25  23.100
Thur No         45  17.113111  41.19   7.51  15.950
     Yes        17  19.190588  43.11  10.34  16.470

In [115]:
# Passing a list of tuples
ftuples = [("purata", "mean"), ("variasi", "var")]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
               purata   variasi     purata     variasi
day  smoker                                           
Fri  No      0.179740  0.001557  18.420000   25.596333
     Yes     0.216293  0.006011  16.813333   82.562438
Sat  No      0.190412  0.003437  19.661778   79.908965
     Yes     0.179833  0.008010  21.276667  101.387535
Sun  No      0.193617  0.003636  20.506667   66.099980
     Yes     0.322021  0.289509  24.120000  109.046044
Thur No      0.193424  0.003143  17.113111   59.625081
     Yes     0.198508  0.003268  19.190588   69.808518

In [116]:
# Apply different functions using a dictionary
grouped.agg({"tip" : "max", "size" : "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [117]:
# Another fancy way
grouped.agg({"total_bill" : ["min", "max", "mean", "std"], "size" : "sum"})

total_bill                              size
                   min    max       mean        std  sum
day  smoker                                             
Fri  No          12.46  22.75  18.420000   5.059282    9
     Yes          5.75  40.17  16.813333   9.086388   31
Sat  No           7.25  48.33  19.661778   8.939181  115
     Yes          3.07  50.81  21.276667  10.069138  104
Sun  No           8.77  48.17  20.506667   8.130189  167
     Yes          7.25  45.35  24.120000  10.442511   49
Thur No           7.51  41.19  17.113111   7.721728  112
     Yes         10.34  43.11  19.190588   8.355149   40

## **Returning Aggregated Data Without Row Indexes**
- Using `as_index`=False/True

In [120]:
# Disable index
tips.groupby(["day", "smoker"], as_index=False)[['total_bill', 'tip', 'size', 'tip_pct']].mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.179740
1,Fri,Yes,16.813333,2.714000,2.066667,0.216293
2,Sat,No,19.661778,3.102889,2.555556,0.190412
3,Sat,Yes,21.276667,2.875476,2.476190,0.179833
4,Sun,No,20.506667,3.167895,2.929825,0.193617
5,Sun,Yes,24.120000,3.516842,2.578947,0.322021
6,Thur,No,17.113111,2.673778,2.488889,0.193424
7,Thur,Yes,19.190588,3.030000,2.352941,0.198508


# **11.3 Apply: General split-apply-combine**

In [137]:
# Define a function to select the top five tip_pct values by group
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [138]:
# Overwrite the default value
top(tips, n=6, column="total_bill")

,total_bill,tip,smoker,day,time,size,tip_pct
170,50.81,10.00,Yes,Sat,Dinner,3,0.245038
212,48.33,9.00,No,Sat,Dinner,4,0.228833
59,48.27,6.73,No,Sat,Dinner,4,0.162013
156,48.17,5.00,No,Sun,Dinner,6,0.115821
182,45.35,3.50,Yes,Sun,Dinner,3,0.083632
102,44.30,2.50,Yes,Sat,Dinner,3,0.059809


In [139]:
# Apply function on certain selected columns
tips.groupby("smoker")[['total_bill', 'tip', 'size', 'tip_pct']].apply(top)

total_bill   tip  size   tip_pct
smoker                                      
No     232       11.61  3.39     2  0.412409
       149        7.51  2.00     2  0.362976
       51        10.29  2.60     2  0.338101
       185       20.69  5.00     5  0.318674
       88        24.71  5.85     2  0.310180
Yes    172        7.25  5.15     2  2.452381
       178        9.60  4.00     2  0.714286
       67         3.07  1.00     1  0.483092
       183       23.17  6.50     4  0.389922
       109       14.31  4.00     2  0.387973

In [140]:
# Overwrite the default values
tips.groupby("smoker")[['total_bill', 'tip', 'size', 'tip_pct']].apply(top, n=3, column="total_bill")

total_bill    tip  size   tip_pct
smoker                                       
No     212       48.33   9.00     4  0.228833
       59        48.27   6.73     4  0.162013
       156       48.17   5.00     6  0.115821
Yes    170       50.81  10.00     3  0.245038
       182       45.35   3.50     3  0.083632
       102       44.30   2.50     3  0.059809

In [141]:
# Earlier example
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.192237,0.057665,0.060217,0.158622,0.184308,0.227015,0.412409
Yes,93.0,0.218176,0.254295,0.036955,0.119534,0.181818,0.242326,2.452381


In [145]:
# Unstack() -> from wide to vertical
result.unstack("smoker")

smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.192237
       Yes         0.218176
std    No          0.057665
       Yes         0.254295
min    No          0.060217
       Yes         0.036955
25%    No          0.158622
       Yes         0.119534
50%    No          0.184308
       Yes         0.181818
75%    No          0.227015
       Yes         0.242326
max    No          0.412409
       Yes         2.452381
dtype: float64

## **Filling Missing Values with Group-Specific Values**

In [147]:
# Create an mock data
rng = np.random.default_rng(seed=12345)
s =pd.Series(rng.standard_normal(6))
s

,0
0,-1.423825
1,1.263728
2,-0.870662
3,-0.259173
4,-0.075343
5,-0.740885


In [148]:
# Skip of 2
s[::2] = np.nan
s

,0
0,NaN
1,1.263728
2,NaN
3,-0.259173
4,NaN
5,-0.740885


In [149]:
# Fill NA with mean
s.fillna(s.mean())

,0
0,0.087890
1,1.263728
2,0.087890
3,-0.259173
4,0.087890
5,-0.740885


In [130]:
# Another example data
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(rng.standard_normal(8), index=states)
data

,0
Ohio,-1.367793
New York,0.648893
Vermont,0.361058
Florida,-1.952863
Oregon,2.347410
Nevada,0.968497
California,-0.759387
Idaho,0.902198


In [131]:
# Insert NaN value into the series
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

,0
Ohio,-1.367793
New York,0.648893
Vermont,NaN
Florida,-1.952863
Oregon,2.347410
Nevada,NaN
California,-0.759387
Idaho,NaN


In [132]:
# Extract size
data.groupby(group_key).size()

,0
East,4
West,4


In [133]:
# Using count function
data.groupby(group_key).count()

,0
East,3
West,2


In [134]:
# Fill the NA values using the group means
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -1.367793
      New York      0.648893
      Vermont      -0.890588
      Florida      -1.952863
West  Oregon        2.347410
      Nevada        0.794011
      California   -0.759387
      Idaho         0.794011
dtype: float64

In [135]:
# With predefined fill values
fill_values = {"East": 0.5, "West": -1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

In [136]:
# Filling in the NA with predefined value
data.groupby(group_key).apply(fill_func)

East  Ohio         -1.367793
      New York      0.648893
      Vermont       0.500000
      Florida      -1.952863
West  Oregon        2.347410
      Nevada       -1.000000
      California   -0.759387
      Idaho        -1.000000
dtype: float64

# **11.4 Group Transforms and “Unwrapped” GroupBys**

In [150]:
# An example data
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [171]:
# Group means by key
g = df.groupby('key')['value']
g.mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [195]:
# Define a mean function
def get_mean(group):
    return group.mean()

In [196]:
# Transform into a Series of mean data
g.transform(get_mean)

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [192]:
# Another way - this is faster using default function
g.transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [197]:
# Transform works with functions that return Series,
# but the result must be the same size as the input
def times_two(kump):
    return kump * 2

g.transform(times_two)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [198]:
# A normalization function
def normalize(x):
    return (x - x.mean()) / x.std()

In [200]:
# Method 1 - faster
g.transform(normalize)

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


In [201]:
# Method 2: apply - slower
# index of the resulting Series will be a MultiIndex
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [178]:
# Unwrapped group operations - doing arithmetic between the outputs of
# multiple GroupBy operations
# often much faster than a general apply function
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


# **11.5 Pivot Tables and Cross-Tabulation**
A ***pivot table*** is a ***data summarization*** tool

In [213]:
# From previous tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886
3,23.68,3.31,No,Sun,Dinner,2,0.162494
4,24.59,3.61,No,Sun,Dinner,4,0.172069
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.256166
240,27.18,2.00,Yes,Sat,Dinner,2,0.079428
241,22.67,2.00,Yes,Sat,Dinner,2,0.096759
242,17.82,1.75,No,Sat,Dinner,2,0.108899


In [214]:
# Getting mean for size and tip_pct columns
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.162612  0.202545
       Sat   2.555556  2.476190  0.190412  0.179833
       Sun   2.929825  2.578947  0.193617  0.322021
       Thur  2.000000       NaN  0.190114       NaN
Lunch  Fri   3.000000  1.833333  0.231125  0.236915
       Thur  2.500000  2.352941  0.193499  0.198508

In [215]:
# Use aggfunc = len for count
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [205]:
# pass NA with a specific value using fill_value
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=-999)

day                        Fri         Sat         Sun        Thur
time   size smoker                                                
Dinner 1    No     -999.000000    0.160000 -999.000000 -999.000000
            Yes    -999.000000    0.483092 -999.000000 -999.000000
       2    No        0.162612    0.198319    0.206535    0.190114
            Yes       0.211180    0.178877    0.400522 -999.000000
       3    No     -999.000000    0.183870    0.182962 -999.000000
            Yes    -999.000000    0.176599    0.183278 -999.000000
       4    No     -999.000000    0.177734    0.175289 -999.000000
            Yes       0.133465    0.147074    0.254373 -999.000000
       5    No     -999.000000 -999.000000    0.263344 -999.000000
            Yes    -999.000000    0.119284    0.070274 -999.000000
       6    No     -999.000000 -999.000000    0.115821 -999.000000
Lunch  1    No     -999.000000 -999.000000 -999.000000    0.222087
            Yes       0.288288 -999.000000 -999.000000 -999.000000
       2    No     -999.000000 -999.000000 -999.000000    0.201503
            Yes       0.226641 -999.000000 -999.000000    0.191197
       3    No        0.231125 -999.000000 -999.000000    0.092162
            Yes    -999.000000 -999.000000 -999.000000    0.257941
       4    No     -999.000000 -999.000000 -999.000000    0.161573
            Yes    -999.000000 -999.000000 -999.000000    0.186592
       5    No     -999.000000 -999.000000 -999.000000    0.138160
       6    No     -999.000000 -999.000000 -999.000000    0.211191

## **Cross-Tabulations: Crosstab**
A ***special pivot table*** to compute ***group frequencies***

In [206]:
from io import StringIO

In [218]:
# Data consists of strings
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

### **`sep="\s+"`**
- "one or more whitespace characters"

In [219]:
# Create a table
data = pd.read_table(StringIO(data), sep="\s+") #used to seperate bnyk whitespace

In [220]:
# Display the data content
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [221]:
# Crosstab example 1 - find group frequencies
# Summarize by nationality and handedness
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [224]:
# Tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886
3,23.68,3.31,No,Sun,Dinner,2,0.162494
4,24.59,3.61,No,Sun,Dinner,4,0.172069
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.256166
240,27.18,2.00,Yes,Sat,Dinner,2,0.079428
241,22.67,2.00,Yes,Sat,Dinner,2,0.096759
242,17.82,1.75,No,Sat,Dinner,2,0.108899


In [225]:
# Crosstab example 2
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

# That's all for the day :)